In [1]:
from utils import getNeighbours, manhattanDistance, BinaryHeap
import math
from collections import deque
import copy

In [2]:
a = [
    ['*', '*', '*', '*', '*', '*', '*', '*', '*', '*'],
    ['*', 'S', '*', '*', '*', ' ', ' ', ' ', ' ', '*'],
    ['*', ' ', '*', '*', '*', ' ', '*', ' ', ' ', '*'],
    ['*', ' ', '*', '*', '*', ' ', '*', '*', ' ', '*'],
    ['*', ' ', '*', '*', ' ', ' ', '*', '*', ' ', '*'],
    ['*', ' ', '*', '*', ' ', ' ', '*', '*', ' ', '*'],
    ['*', ' ', ' ', ' ', ' ', '*', '*', '*', ' ', '*'],
    ['*', '*', '*', ' ', ' ', '*', '*', '*', 'T', '*'],
    ['*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
]
start = (1, 1)
goal = (7, 8)

known = [[manhattanDistance((j, i), goal) for i in range(len(a[0]))] for j in range(len(a))]
known

[[15, 14, 13, 12, 11, 10, 9, 8, 7, 8],
 [14, 13, 12, 11, 10, 9, 8, 7, 6, 7],
 [13, 12, 11, 10, 9, 8, 7, 6, 5, 6],
 [12, 11, 10, 9, 8, 7, 6, 5, 4, 5],
 [11, 10, 9, 8, 7, 6, 5, 4, 3, 4],
 [10, 9, 8, 7, 6, 5, 4, 3, 2, 3],
 [9, 8, 7, 6, 5, 4, 3, 2, 1, 2],
 [8, 7, 6, 5, 4, 3, 2, 1, 0, 1],
 [9, 8, 7, 6, 5, 4, 3, 2, 1, 2]]

In [190]:
class Node:
    
    def __init__(self, point, f, parent=None):
        self.point = point
        self.f = f
        self.parent = parent
    
    def __lt__(self, other):
        if self == other:
            return self.f < other.f

    def __eq__(self, o):
        return self.point == o.point
    
    def __gt__(self, other):
        if self == other:
            return self.f > other.f
    
    def __str__(self):
        return str(self.point) + " " + str(self.f)
    
    def getNeighbours(self, grid_dim):
        neighbours = []
        pos = self.point
        if pos[0] > 0:
            neighbours.append((pos[0] - 1, pos[1]))
        if pos[0] + 1< grid_dim[0] -1:
            neighbours.append((pos[0] + 1, pos[1]))
        if pos[1] > 0:
            neighbours.append((pos[0], pos[1] - 1))
        if pos[1]  < grid_dim[1] -1:
            neighbours.append((pos[0], pos[1]+1))
        
        return neighbours

In [191]:
Node(point=(1, 9), f=0).getNeighbours(grid_dim=(9, 10))

[(0, 9), (2, 9), (1, 8)]

In [192]:


# Function to implement A* search algorithm


class AStar:
    def __init__(self, known, start, goal):
        """
            Function that calculates the shortest path from start to goal given a grid and the known world
            known contains h value for each cell
            If a cell is an obstacle, the h value becomes infinity, 

            Args:
                known (list): known grid
                start (tuple): start coordinates
                goal (tuple): goal coordinates
        """    
        self.grid_dim = (len(known[0]), len(known))
        self.openList = queue.PriorityQueue()   
        self.closedList = []
        self.known = known
        self.openList.put((0, start, []))
        self.path = []
        # g Values
        

    def run(self):    
        gMatrix = [[0 for i in range(len(self.known[0]))] for j in range(len(self.known))]
        while self.openList.empty() == False:
        # for i in range(20):
            # print("Open List is", self.openList.queue)
            # print("Closed List is", self.closedList)
            _, current, history = self.openList.get()
            # print("Agent is at", current)
            neighbours = getNeighbours(self.grid_dim, current)
            
            # Observe surroundings
            for i in range(len(neighbours)):
                # print(neighbours[i])
                if neighbours[i] == goal:
                    # print("Found goal")
                    #TODO: Return path
                    history.extend([current, goal])
                    self.path = history
                    return sorted(set(self.path), key=self.path.index)
                    # return self.path
                

                g = gMatrix[current[0]][current[1]] + 1
                h = known[neighbours[i][0]][neighbours[i][1]]
                f = g + h
                if f == math.inf:
                    continue
                present, f_check = self.checkCellInOpenList(neighbours[i])
                # print("Checking open list")
                if present:
                    if f >= f_check:
                        continue
                
                present, f_check = self.checkCellInClosedList(neighbours[i])
                # print("Checking closed list")
                if present:
                    # print(neighbours[i], "present in closed list", f, f_check)
                    if f >= f_check:
                        # print("Adding ", neighbours[i], "with f =", f_check, f,"to open list")
                        continue    
                # print("Adding ", neighbours[i], "with f =", f,"to open list")
                history.append(current)
                self.openList.put((f, neighbours[i], history))
                gMatrix[neighbours[i][0]][neighbours[i][1]] = g    
                
            self.closedList.append((gMatrix[current[0]][current[1]], current))

    
    def checkCellInOpenList(self, pos):
        for i in self.openList.queue:
            if i[1] == pos:
                return True, i[0]
        return False, 0
    
    def checkCellInClosedList(self, pos):
        for i in self.closedList:
            if i[1] == pos:
                return True, i[0]
        return False, 0
    
    
    


In [290]:
class RepeatedAStar:
    def __init__(self, grid, start, goal):
        self.counter = 0
        self.search = [[0 for i in range(len(grid[0]))] for j in range(len(grid))]
        self.grid = grid
        self.grid_dim = (len(grid), len(grid[0]), )
        self.start, self.goal = start, goal
        self.g = [[0 for i in range(len(grid[0]))] for j in range(len(grid))]
        # print(self.g)
        self.h = [[manhattanDistance((i, j), goal) for i in range(len(grid[0]))] for j in range(len(grid))]
        self.current = Node(point=start, f=self.h[start[0]][start[1]])
        
    def run(self):
        
        while self.current.point != self.goal:
            print(self.current)
            
            
            self.counter += 1
            self.g[self.start[0]][self.start[1]] = 0
            self.search[self.current.point[0]][self.current.point[1]] = self.counter
            self.g[self.goal[0]][self.goal[1]] = math.inf
            self.search[self.goal[0]][self.goal[1]] = self.counter
            
            self.openList = BinaryHeap()
            self.closedList = []
            self.openList.push(Node(point=self.current.point, f=self.g[self.current.point[0]][self.current.point[1]] + self.h[self.current.point[0]][self.current.point[1]]))
            
            aStar = self.aStar()
            
            #TODO: Implement Repeated A* search
            
            if self.openList.size == 0:
                print("Cannot reach goal")
                return
            
            if self.openList.peek().point == self.goal:
                # print("Found goal")
                #TODO: find reverse path and move towards goal till you reach the goal or cost increases
                
                curr = self.openList.peek()
                path = deque()
                
                
                while curr.parent:
                    path.appendleft(curr)
                    curr = curr.parent
                h = copy.deepcopy(self.h)
                
                        
                # Update the current position of the agent
                # return path
                # return h, self.h
                while self.current.point != self.goal:
                    # # Update Obstacles that the agent can see
                    # curr_neighbours = self.current.getNeighbours(self.grid_dim)
                    # for i in range(len(curr_neighbours)):
                    #     if self.grid[curr_neighbours[i][0]][curr_neighbours[i][1]] == "*":
                    #         h[curr_neighbours[i][0]][curr_neighbours[i][1]] = math.inf
                    # if h[self.current.point[0]][self.current.point[1]] == math.inf:
                    #     break
                    next_block = path.popleft()
                    if self.grid[next_block.point[0]][next_block.point[1]] == "*":
                        break
                    self.current = path.popleft()
                    print(self.current)
                    # if h[self.current.point[0]][self.current.point[1]] == math.inf:
                    #     print("Found obstacle")
                    #     break
                
                
                    
        
        
        
    def aStar(self):
        while self.g[self.goal[0]][self.goal[1]] > self.openList.peek().f:
            # print("Open List is", self.openList.heap)
            current = self.openList.pop()
            # print("Analyzing", current.point)
            self.closedList.append(current)
            neighbours = current.getNeighbours(self.grid_dim)
            # print(neighbours)
            for neighbour in neighbours:
                # print("Neighbour is", neighbour)
                if self.g[neighbour[0]][neighbour[1]] < self.counter:
                    self.g[neighbour[0]][neighbour[1]] = math.inf
                    self.search[neighbour[0]][neighbour[1]] = self.counter

                if self.g[neighbour[0]][neighbour[1]] > self.g[current.point[0]][current.point[1]] + 1:         # c(s, a) = 1
                    self.g[neighbour[0]][neighbour[1]] = self.g[current.point[0]][current.point[1]] + 1
                    new = Node(point=neighbour, f=self.g[neighbour[0]][neighbour[1]] + self.h[neighbour[0]][neighbour[1]], parent=current)
                    
                    if new in self.openList.heap[1:]:
                        self.openList.remove(new)
                    self.openList.push(new)
                    # print("Added", neighbour, "to open list")
                    
                    

In [292]:
r = RepeatedAStar(a, start, goal)
r.run()

(1, 1) 13
(1, 2) 13
(1, 2) 13
(1, 2) 13


IndexError: list index out of range

In [282]:
len(path)

29

In [281]:
for x in list(path):
    print(x.point)

(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(2, 9)
(3, 9)
(4, 9)
(5, 9)
(5, 8)
(5, 7)
(5, 6)
(5, 5)
(5, 4)
(5, 3)
(5, 2)
(5, 1)
(6, 1)
(7, 1)
(7, 2)
(7, 3)
(7, 4)
(7, 5)
(7, 6)
(7, 7)
(7, 8)


In [286]:
path[0]

[[15, inf, 13, 12, 11, 10, 9, 8, 9, 10],
 [inf, 13, inf, 11, 10, 9, 8, 7, 8, 9],
 [13, 12, 11, 10, 9, 8, 7, 6, 7, 8],
 [12, 11, 10, 9, 8, 7, 6, 5, 6, 7],
 [11, 10, 9, 8, 7, 6, 5, 4, 5, 6],
 [10, 9, 8, 7, 6, 5, 4, 3, 4, 5],
 [9, 8, 7, 6, 5, 4, 3, 2, 3, 4],
 [8, 7, 6, 5, 4, 3, 2, 1, 2, 3],
 [7, 6, 5, 4, 3, 2, 1, 0, 1, 2]]

In [287]:
path[1]

[[15, 14, 13, 12, 11, 10, 9, 8, 9, 10],
 [14, 13, 12, 11, 10, 9, 8, 7, 8, 9],
 [13, 12, 11, 10, 9, 8, 7, 6, 7, 8],
 [12, 11, 10, 9, 8, 7, 6, 5, 6, 7],
 [11, 10, 9, 8, 7, 6, 5, 4, 5, 6],
 [10, 9, 8, 7, 6, 5, 4, 3, 4, 5],
 [9, 8, 7, 6, 5, 4, 3, 2, 3, 4],
 [8, 7, 6, 5, 4, 3, 2, 1, 2, 3],
 [7, 6, 5, 4, 3, 2, 1, 0, 1, 2]]

In [293]:
r.g

[[2, 3, 4, 3, 4, 5, 6, 7, 8, 9],
 [3, 4, 3, 4, 3, 4, 5, 6, 7, 8],
 [4, 3, 4, 3, 4, 5, 6, 7, 8, 9],
 [5, 4, 5, 6, 7, 8, 9, 10, 9, 10],
 [6, 5, 6, 7, 8, 9, 10, 11, 10, 11],
 [7, 8, 9, 10, 11, 12, 11, 12, 13, 12],
 [8, 9, 10, 11, 12, 13, 12, 13, 14, 13],
 [21, 22, 21, 22, 15, 14, 15, 14, inf, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]